In [31]:
# Import packages
from dash import Dash, html, dash_table, dcc, callback, Output, Input
import pandas as pd
import plotly.express as px
import dash_bootstrap_components as dbc

# Incorporate data
df = pd.read_csv('https://github.com/feliphearaujo/dashboard_vendas_dash/blob/main/base_dados.csv?raw=true',encoding = 'latin-1')

# Initialize the app

external_stylesheets = [dbc.themes.SLATE]

app = Dash(__name__, external_stylesheets=external_stylesheets)

# App layout
app.layout = dbc.Container([
    dbc.Row([
        html.Div('DASHBOARD VENDAS', className=".text-light-emphasis text-center fs-3",style={
                            'text-align': 'center',
                            'font-weight': 'bold',
                            'font-size': '60px',
                            'color': '#FFFF',
                            'margin-top':'10px'
        })
    ]),
    html.Hr(style={'border-color': '#FFFF','height': '15px','margin-bottom': '10px'}),
    dbc.Row(
        [
            dbc.Col(
                md=6,
                children=[
                    dbc.Row(
                        style={'margin': '5px'},children=[
                        html.H4('Total de Vendas / Mês',style={
                            'text-align': 'center',
                            'font-weight': 'bold',
                            'font-size': '30px',
                            'color': '#FFFF',
                            'background-color': '#212428',
                            'padding':'10px',
                            'margin':'5px'
                        }),
                        ]
                    )
                ]
            ),
            dbc.Col(
                md=6,
                children=[
                    dbc.Row(
                        style={'margin': '5px'},children=[
                        html.H4('Total de Vendas / Representante',style={
                            'text-align': 'center',
                            'font-weight': 'bold',
                            'font-size': '30px',
                            'color': '#FFFF',
                            'background-color': '#212428',
                            'padding':'10px',
                            'margin':'5px'
                        }),
                        ]
                    )
                ]
            ),
            dbc.Row(

            )
        ]
        )


    

], fluid=True)

# Run the app
if __name__ == '__main__':
    app.run(debug=True)

In [ ]:
global_df = df.copy()

global_df['Data_Pedido'] = pd.to_datetime(global_df['Data_Pedido'])
global_df['Ano'] = global_df['Data_Pedido'].dt.year
global_df['Mes'] = global_df['Data_Pedido'].dt.month_name()